In [1]:
import numpy as np
import pandas as pd

# Configuración
n_matrices = 10000  # 10,000 matrices
n_runs = 10         # 10 ejecuciones
percentiles_to_calculate = [0.1, 1, 2, 3, 4, 5, 10, 20, 25, 50]  # Percentiles

# Función para generar matrices fuzzy aleatorias con diagonal 0.5
def generate_random_fuzzy_matrices(n_matrices, n):
    matrices = np.random.uniform(0.0001, 0.9999, size=(n_matrices, n, n))
    for i in range(n):
        matrices[:, i, i] = 0.5
    return matrices

# Función para calcular la no reciprocidad
def calculate_non_reciprocity(norm, a):
    n = a.shape[1]  # tamaño de la matriz (2, 3, ..., 8)
    
    # Extraer todos los pares únicos (i < j)
    indices = [(i, j) for i in range(n) for j in range(i+1, n)]

    ln_list = []
    for idx in indices:
        i, j = idx
        a_ij = a[:, i, j]
        a_ji = a[:, j, i]
        
        t = (a_ij * a_ji) / ((1 - a_ij) * (1 - a_ji))
        t = np.maximum(t, 1e-10)  # Evitar log(0) o negativos
        
        ln_t = np.abs(np.log(t))
        ln_list.append(ln_t)

    # ln_list es una lista de arrays -> apilamos
    ln_list = np.stack(ln_list, axis=1)  # (n_matrices, n_pairs)

    if norm == 1:
        non_reciprocity = np.sum(ln_list, axis=1)
    elif norm == 2:
        non_reciprocity = np.sqrt(np.sum(ln_list ** 2, axis=1))
    elif norm == np.inf:
        non_reciprocity = np.max(ln_list, axis=1)
    else:
        raise ValueError("Norma no soportada.")
    
    return non_reciprocity

# Función para generar los resultados para un tamaño y norma dados
def generate_results_for_norm(n, norm, filename):
    results_list = []

    for percentile in percentiles_to_calculate:
        run_percentiles = []
        for run in range(n_runs):
            a = generate_random_fuzzy_matrices(n_matrices, n)
            non_reciprocity = calculate_non_reciprocity(norm, a)
            run_percentile_value = np.percentile(non_reciprocity, percentile)
            run_percentiles.append(run_percentile_value)

        average_percentile = np.mean(run_percentiles)

        row = {"percentile": percentile}
        for i in range(n_runs):
            row[f"Run {i+1}"] = run_percentiles[i]
        row["Avg"] = average_percentile

        results_list.append(row)

    results = pd.DataFrame(results_list)
    results.to_excel(filename, index=False)

# Ahora, generar para n=2 hasta n=8 y normas p=1, 2, inf
for n in range(2, 9):  # 2 to 8 inclusive
    generate_results_for_norm(n, 1, f"fuzzy_{n}x{n}_p1.xlsx")
    generate_results_for_norm(n, 2, f"fuzzy_{n}x{n}_p2.xlsx")
    generate_results_for_norm(n, np.inf, f"fuzzy_{n}x{n}_pinf.xlsx")


In [2]:
import numpy as np
import pandas as pd

# Configuración
n_matrices = 10000  # 10,000 matrices
n_runs = 10         # 10 ejecuciones
percentiles_to_calculate = [0.1, 1, 2, 3, 4, 5, 10, 20, 25, 50]  # Percentiles

# Función para generar matrices aditivas aleatorias en [-100, 100]
def generate_random_additive_matrices(n_matrices, n):
    matrices = np.random.uniform(-100, 100, size=(n_matrices, n, n))
    return matrices

# Función para calcular la no reciprocidad para matrices aditivas
def calculate_non_reciprocity_additive(norm, a):
    n = a.shape[1]  # tamaño de la matriz (2, 3, ..., 8)
    
    # Extraer todos los pares únicos (i < j)
    indices = [(i, j) for i in range(n) for j in range(i+1, n)]

    ln_list = []
    for idx in indices:
        i, j = idx
        a_ij = a[:, i, j]
        a_ji = a[:, j, i]
        
        # Relación para aditivas
        t = a_ij / (-a_ji)
        t = np.maximum(np.abs(t), 1e-10)  # Evitar log(0) o negativos
        
        ln_t = np.abs(np.log(t))
        ln_list.append(ln_t)

    # ln_list es una lista de arrays -> apilamos
    ln_list = np.stack(ln_list, axis=1)  # (n_matrices, n_pairs)

    if norm == 1:
        non_reciprocity = np.sum(ln_list, axis=1)
    elif norm == 2:
        non_reciprocity = np.sqrt(np.sum(ln_list ** 2, axis=1))
    elif norm == np.inf:
        non_reciprocity = np.max(ln_list, axis=1)
    else:
        raise ValueError("Norma no soportada.")
    
    return non_reciprocity

# Función para generar los resultados para un tamaño y norma dados
def generate_results_for_norm(n, norm, filename):
    results_list = []

    for percentile in percentiles_to_calculate:
        run_percentiles = []
        for run in range(n_runs):
            a = generate_random_additive_matrices(n_matrices, n)
            non_reciprocity = calculate_non_reciprocity_additive(norm, a)
            run_percentile_value = np.percentile(non_reciprocity, percentile)
            run_percentiles.append(run_percentile_value)

        average_percentile = np.mean(run_percentiles)

        row = {"percentile": percentile}
        for i in range(n_runs):
            row[f"Run {i+1}"] = run_percentiles[i]
        row["Avg"] = average_percentile

        results_list.append(row)

    results = pd.DataFrame(results_list)
    results.to_excel(filename, index=False)

# Ahora, generar para n=2 hasta n=8 y normas p=1, 2, inf
for n in range(2, 9):  # 2 to 8 inclusive
    generate_results_for_norm(n, 1, f"additive_{n}x{n}_p1.xlsx")
    generate_results_for_norm(n, 2, f"additive_{n}x{n}_p2.xlsx")
    generate_results_for_norm(n, np.inf, f"additive_{n}x{n}_pinf.xlsx")
